#GROQ


In [1]:
!pip install groq
import os
key="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"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 3.1 MB/s eta 0:00:00


In [2]:
from groq import Groq
os.environ["GROQ_API_KEY"] = key
client = Groq()
completion = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[
        {
            "role": "user",
            "content": "what is cricket"
        }
    ],
    temperature=1,
    max_completion_tokens=1024,
    top_p=1,
    stream=True,
    stop=None,
)
response_text = ""
for chunk in completion:
    response_text += chunk.choices[0].delta.content or ""


In [3]:
type(response_text)

str

In [4]:
student_description = "sunny savita is a student of computer science at IIT delhi. He is an indian and has a 8.5 GPA. Sunny is known for his programming skills and is an active member of the college's AI Club. He hopes to pursue a career in artificial intelligence after graduating."

In [5]:
# A simple prompt to extract information from "student_description" in a JSON format.
prompt = f'''
Please extract the following information from the given text and return it as a JSON object:

name
college
grades
club

This is the body of text to extract the information from:
{student_description}
'''

In [6]:
prompt

"\nPlease extract the following information from the given text and return it as a JSON object:\n\nname\ncollege\ngrades\nclub\n\nThis is the body of text to extract the information from:\nsunny savita is a student of computer science at IIT delhi. He is an indian and has a 8.5 GPA. Sunny is known for his programming skills and is an active member of the college's AI Club. He hopes to pursue a career in artificial intelligence after graduating.\n"

In [7]:

completion2 = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ],
)



In [8]:
output=completion2.choices[0].message.content

In [9]:
print(output)

Here's the extracted information in a JSON object:

```json
{
    "name": "Sunny Savita",
    "college": "IIT Delhi",
    "grades": "8.5 GPA",
    "club": "AI Club"
}
```


#Example of function calling


In [10]:
function_descriptions = [
    {
        "name": "get_flight_info",
        "description": "Get flight information between two locations",
        "parameters": {
            "type": "object",
            "properties": {
                "loc_origin": {
                    "type": "string",
                    "description": "The departure airport, e.g. DEL",
                },
                "loc_destination": {
                    "type": "string",
                    "description": "The destination airport, e.g. MUM",
                },
            },
            "required": ["loc_origin", "loc_destination"],
        },
    }
]

In [11]:
user_prompt = "When's the next flight from new lahore to karachi?"

In [12]:
completion2 = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[
        {
            "role": "user",
            "content": user_prompt

        }
    ],
    functions=function_descriptions,
    function_call="auto",
)


In [13]:
completion2.choices[0].message.function_call.arguments

'{"loc_origin": "LHE", "loc_destination": "KHI"}'

In [14]:
import json
from datetime import datetime,timedelta
def get_flight_info(loc_origin, loc_destination):
    """Get flight information between two locations."""

    # Example output returned from an API or database
    flight_info = {
        "loc_origin": loc_origin,
        "loc_destination": loc_destination,
        "datetime": str(datetime.now() + timedelta(hours=2)),
        "airline": "KLM",
        "flight": "KL643",
    }

    return json.dumps(flight_info)

In [15]:
params=json.loads(completion2.choices[0].message.function_call.arguments)

In [16]:
origin = json.loads(completion2.choices[0].message.function_call.arguments).get("loc_origin")
destination = json.loads(completion2.choices[0].message.function_call.arguments).get("loc_destination")

In [17]:
print(origin)

LHE


In [18]:
print(destination)

KHI


In [19]:
completion2.choices[0].message.function_call.name

'get_flight_info'

In [20]:
chosen_function=eval(completion2.choices[0].message.function_call.name)

In [21]:
flight = chosen_function(**params)
print(flight)

{"loc_origin": "LHE", "loc_destination": "KHI", "datetime": "2025-03-29 09:12:15.956352", "airline": "KLM", "flight": "KL643"}


In [22]:
flight_response = json.loads(flight)

In [23]:
completion3 = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[
        {"role": "user", "content": user_prompt},
        {"role": "function", "name": completion2.choices[0].message.function_call.name, "content": flight},
        {"role": "assistant", "content": "Here are your flight details: " + flight}
    ],
    functions=function_descriptions,
    function_call="auto",
)


In [24]:
if completion3.choices[0].message.content is None:
    print("Here are your flight details:", flight_response)
else:
    print(completion3.choices[0].message.content)


 </s><function=get_flight_info{"loc_origin": "LHE", "loc_destination": "KHI"}</function>


#Langchain

In [25]:
!pip install langchain-groq

In [26]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

In [27]:
%env GROQ_API_KEY=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

env: GROQ_API_KEY=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


In [28]:
prompt1="Can you tell me number of countries in asia? just give me top 10 countires name"

In [29]:
prompt2="can you tell me the capital of pakistan"

In [30]:
chat = ChatGroq(temperature=0, model_name="llama-3.3-70b-versatile")

In [31]:
from langchain.prompts import PromptTemplate

In [32]:
prompt_template_name=PromptTemplate(
    input_variables=["Country"],
    template="can you tell me capital of {Country}?"
)

In [33]:
prompt1=prompt_template_name.format(Country="Pakistan")

In [34]:
promp2=prompt_template_name.format(Country="India")

In [35]:
prompt3="Can you tell me who won recent cricket champions trophy??"

In [36]:
system = "You are a helpful assistant."
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | chat
output1=chain.invoke({prompt1})
output2=chain.invoke({promp2})
output3=chain.invoke({prompt3})

In [37]:
print(output1.content)
print(output2.content)
print(output3.content)


The capital of Pakistan is Islamabad.
The capital of India is **New Delhi**.
The most recent ICC Champions Trophy was held in 2017. Pakistan won the tournament by defeating India in the final with a score of 338/4, chasing India's 158 all out in just 30.3 overs, and then Pakistan made 338/4 in 50 overs, winning the match by 180 runs. Fakhar Zaman was the player of the match, scoring 114 runs. This was Pakistan's first ICC Champions Trophy title.


#Agents


In [38]:
!pip install google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32009 sha256=d090654627492180a68d7cdcc1853262693dce330720c912ca3afb14635b9c0b
  Stored in directory: /root/.cache/pip/wheels/6e/42/3e/aeb691b02cb7175ec70e2da04b5658d4739d2b41e5f73cd06f
Successfully built google-search-results


In [39]:
serp_key="6949952e6ede49f114caff1a02612ec0bd7f3b907ff2e270f50d6088becc6f2f"

In [40]:
!pip install google-search-results
!pip install -U langchain langchain-community langchain-groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00


In [41]:
from langchain_groq import ChatGroq
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.tools import Tool

In [42]:
groq_key = "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"
serp_key = "6949952e6ede49f114caff1a02612ec0bd7f3b907ff2e270f50d6088becc6f2f"

In [43]:
llm = ChatGroq(api_key=groq_key, model_name="llama-3.3-70b-versatile")

In [44]:
tools = load_tools(["serpapi"], serpapi_api_key=serp_key)

In [45]:
agent = initialize_agent(
    tools=tools,llm=llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True
)

<ipython-input-45-e4c86c871b89>:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [46]:
response = agent.invoke("Can you tell me who won recent cricket champions trophy??")
print(response)



> Entering new AgentExecutor chain...
To find out who won the recent cricket champions trophy, I should first search for the latest information on the cricket champions trophy.

Action: Search
Action Input: recent cricket champions trophy winner
Observation: {'title': 'ICC Champions Trophy', 'games': [{'tournament': 'ICC Champions Trophy', 'venue': 'Dubai International Stadium', 'venue_kgmid': '/m/054kpj5', 'date': 'Mar 9', 'teams': [{'name': 'New Zealand', 'score': '251/7 (50)', 'kgmid': '/m/02fbb5', 'thumbnail': 'https://serpapi.com/searches/67e79d8abdd7f4e5cda32e0e/images/e88f6ee25bcb67ed0f1d2a25c58c0e6fc7966e79112104a064acf59b0a87df2825ba1b80a588cb3372911f878bf123da.png'}, {'name': 'India', 'score': '254/6 (49)', 'kgmid': '/m/024nj1', 'thumbnail': 'https://serpapi.com/searches/67e79d8abdd7f4e5cda32e0e/images/e88f6ee25bcb67ed0f1d2a25c58c0e6fc7966e79112104a06139da32417a52896c0e21f939cc949daac0ef8cb3c8739c.png'}], 'status': 'IND won by 4 wickets (6 balls left)'}, {'tournament': 'ICC

#Chains


In [47]:
# Example_no_1(BASIC)
from langchain.prompts import PromptTemplate
prompt= PromptTemplate(input_variables="Product",template="what is the good name for the company that make {Product} give only one name")
prompt1=prompt.format(Product="wine")


In [48]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt,verbose=True)
result = chain.run({"Product": "wine"})
print(result)



> Entering new LLMChain chain...
Prompt after formatting:
what is the good name for the company that make wine give only one name


<ipython-input-48-69f03da6a67f>:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt,verbose=True)
<ipython-input-48-69f03da6a67f>:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = chain.run({"Product": "wine"})



> Finished chain.
Vinterra


In [49]:
from langchain.chains import LLMChain, SequentialChain

In [50]:
title_prompt = PromptTemplate(
    input_variables=["topic"],
    template="Suggest a catchy blog title for this topic: {topic}"
)
title_chain = LLMChain(
    llm=llm,
    prompt=title_prompt,
    output_key="title"
)
outline_prompt = PromptTemplate(
    input_variables=["title"],
    template="Create a blog outline based on this title: {title}"
)
outline_chain = LLMChain(
    llm=llm,
    prompt=outline_prompt,
    output_key="outline"
)

In [51]:
sequential_chain = SequentialChain(
    chains=[title_chain, outline_chain],
    input_variables=["topic"],
    output_variables=["title", "outline"],
    verbose=True
)
result = sequential_chain.invoke({"topic": "The Future of AI in Healthcare"})
print(result)



> Entering new SequentialChain chain...

> Finished chain.
{'topic': 'The Future of AI in Healthcare', 'title': 'Here are a few suggestions for a catchy blog title on the topic of "The Future of AI in Healthcare":\n\n1. **"Healing Bytes: The AI Revolution in Healthcare"**\n2. **"Medicine Rebooted: How AI is Transforming Healthcare"**\n3. **"The Future of Care: AI\'s Impact on Healthcare"**\n4. **"AI for Better Health: The Next Frontier in Medicine"**\n5. **"Prescribing Progress: The Role of AI in Shaping Healthcare\'s Future"**\n6. **"Smart Medicine: The Convergence of AI and Healthcare"**\n7. **"Revolutionizing Recovery: The Power of AI in Healthcare"**\n8. **"From Code to Cure: The Emerging Role of AI in Healthcare"**\n9. **"Healthcare 2.0: The AI-Driven Future of Medicine"**\n10. **"AI in the Clinic: The Future of Personalized Medicine"**\n\nChoose the one that resonates the most with your blog\'s tone and style, or feel free to modify any of these suggestions to fit your needs.',

#Document Loader


In [52]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 4.7 MB/s eta 0:00:00


In [54]:
# from langchain_community.document_loaders import PyPDFLoader

# loader = PyPDFLoader("/content/Python_100_Questions_and_Answers.pdf")
# pages = loader.load_and_split()

#Memory


In [58]:
from langchain.memory import ConversationBufferMemory
memory=ConversationBufferMemory()
title_prompt = PromptTemplate(
    input_variables=["topic"],
    template="Suggest a catchy blog title for this topic: {topic}"
)
title_chain = LLMChain(
    llm=llm,prompt=title_prompt,memory=memory
)


<ipython-input-58-676f71fbf776>:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory=ConversationBufferMemory()


In [63]:
print(title_chain.run("wines").strip())

Here are a few suggestions for a catchy blog title about wines:

1. **Grape Expectations**: A playful name that references the classic novel "Great Expectations" while incorporating a wine-related pun.
2. **The Wine Wanderer**: This title evokes a sense of adventure and exploration, perfect for a blog that delves into the world of wine.
3. **Vino Vibes**: A fun, laid-back title that captures the relaxed, social atmosphere often associated with wine.
4. **The Wine Diaries**: A personal, introspective title that suggests the blog will be a collection of thoughts, experiences, and insights about wine.
5. **Uncorked**: A simple, attention-grabbing title that references the act of opening a bottle of wine, with a hint of excitement and discovery.
6. **Wine Wise**: A title that implies the blog will be a trusted source of knowledge and expertise about wine.
7. **The Grape Gazette**: A playful, old-fashioned title that suggests a blog that's informative, engaging, and maybe even a little bit 

In [64]:
print(title_chain.run("laptops").strip())

Here are a few ideas for a catchy blog title about laptops:

1. **"The Laptop Lowdown"** - a playful name that implies the blog will give readers the inside scoop on laptops.
2. **"Byte-Sized Reviews"** - a clever title that references the digital world of laptops and suggests the blog will offer concise, bite-sized reviews.
3. **"Laptop Lounge"** - a relaxed, inviting title that implies the blog is a comfortable space to learn about laptops.
4. **"The Laptop Diaries"** - a personal, storytelling-style title that suggests the blog will share the author's experiences and insights about laptops.
5. **"Power Up!"** - a energetic, motivational title that implies the blog will help readers get the most out of their laptops.
6. **"Laptop Life"** - a simple, straightforward title that suggests the blog will explore the ways laptops fit into our daily lives.
7. **"The Tech Trek"** - a title that implies the blog will take readers on a journey of discovery about laptops and technology.
8. **"La

In [68]:
print(title_chain.memory.buffer.strip())

Human: Ai
AI: Here are some catchy blog title ideas for the topic of AI:

1. **"Rise of the Machines: Exploring the Future of AI"**
2. **"AI Insider: Uncovering the Latest Trends and Innovations"**
3. **"The AI Revolution: How Artificial Intelligence is Changing Everything"**
4. **"Byte-Sized Intelligence: The World of Artificial Intelligence"**
5. **"Smart Machines, Smarter Humans: The AI Blog"**
6. **"The AI Chronicles: Stories from the Frontiers of Artificial Intelligence"**
7. **"Intelligent Insights: Unraveling the Mysteries of AI"**
8. **"AI Unplugged: Uncovering the Secrets of Artificial Intelligence"**
9. **"Machine Learning Mastery: The Ultimate AI Blog"**
10. **"The Future is Now: Exploring the Possibilities of AI"**

Which one do you like the most? Or would you like me to come up with more options?
Human: Ai
AI: Here are a few catchy blog title ideas for the topic of AI:

1. **"Machine Mastery: Exploring the Frontiers of Artificial Intelligence"**
2. **"The AI Advantage: How

In [70]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [72]:
memory=ConversationBufferMemory()
converstion=ConversationChain(llm=llm,memory=memory)


<ipython-input-72-6418c4e9364c>:2: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  converstion=ConversationChain(llm=llm,memory=memory)


In [75]:
print(converstion.run("What is 2+2").strip())

We've already established that 2+2 equals 4. As I mentioned earlier, this is a basic arithmetic operation that I've been trained on, and the result is well-defined within the context of Peano axioms. If you're interested, I can provide more information on how this equation is used in various mathematical and real-world applications. For instance, in computer science, the equation 2+2=4 can be used to demonstrate the fundamentals of binary arithmetic, which is the basis for all computer calculations. Would you like to explore some of these applications or is there something else I can help you with?


In [77]:
print(converstion.run("Who won the 1st world cup of cricket").strip())


The first Cricket World Cup was held in 1975 in England, and it was won by the West Indies team, led by Clive Lloyd. The West Indies defeated Australia in the final at Lord's Cricket Ground on June 21, 1975, with a score of 291/8 in 60 overs, beating Australia's 274 in 58.4 overs. The 1975 Cricket World Cup was the inaugural edition of the tournament, featuring eight teams from around the world, and it set the stage for the modern Cricket World Cup. The West Indies' victory in the first tournament marked the beginning of their dominance in international cricket, with the team going on to win the next two World Cups as well, in 1979 and 1983, under the captaincy of Clive Lloyd and then Vivian Richards, respectively. Would you like to know more about the history of the Cricket World Cup or the West Indies team's performance during that era?


In [78]:
print(converstion.run("What is 2+3").strip())

The answer to 2+3 is 5. This is another basic arithmetic operation that I've been trained on, and the result is well-defined within the context of Peano axioms, which I mentioned earlier when we discussed the equation 2+2=4. In fact, the equation 2+3=5 can be derived using the same principles and axioms that we used to prove 2+2=4. If you're interested, I can provide more information on how this equation is used in various mathematical and real-world applications, such as in computer science, physics, or engineering. For instance, in computer programming, the equation 2+3=5 can be used to demonstrate the fundamentals of arithmetic operations in programming languages, such as Java or Python. Would you like to explore some of these applications or is there something else I can help you with?


In [79]:
print(converstion.run("Who was the captain of the cricket team").strip())

You're referring to the West Indies team that won the first Cricket World Cup in 1975. The captain of the West Indies team during that tournament was Clive Lloyd. He was a highly respected and successful captain, leading the team to victory in the 1975 and 1979 Cricket World Cups. Clive Lloyd is considered one of the greatest captains in the history of cricket, known for his exceptional leadership skills, tactical acumen, and ability to bring out the best in his players. Under his captaincy, the West Indies team became a dominant force in international cricket, producing some of the most iconic players of the era, including Vivian Richards, Malcolm Marshall, and Joel Garner. Would you like to know more about Clive Lloyd's career or the West Indies team's performance during the 1970s?


In [80]:
print(converstion.run("Roast me on my inputs").strip())

It seems like you've been providing some fairly straightforward and simple inputs, and I'm happy to roast you on them.

Let's start with the fact that you asked me what 2+2 is, not once, but twice. I mean, come on, did you really forget the answer in the span of a few minutes? It's not like I'm a math teacher or something (although, I did provide a proof using Peano axioms, so maybe I am).

And then, you asked me who won the first Cricket World Cup, which is a pretty basic question for any cricket fan. But hey, I guess it's okay, since you did follow it up with a question about the captain of the winning team. Although, it would've been nice if you'd asked something a bit more challenging, like the average score of the West Indies team during that tournament or something.

But the real kicker is when you asked me what 2+3 is. I mean, seriously, are you trying to test my math skills or something? It's not like I'm a supercomputer or anything (oh wait, I am). Anyway, I'm glad I could pro